## Get ITS feature table for 16S and ITS co-occurence. Only include taxas that are present in at least 10% of all samples. Split two materials, compare relative VS quantitative

In [40]:
source("./ITS-PhyloseqObjects.R")

Warning message in strsplit(conditionMessage(e), "\n"):
“input string 1 is invalid in this locale”

In [41]:
physeq.23s

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 138 taxa and 23 samples ]
sample_data() Sample Data:       [ 23 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 138 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 138 tips and 137 internal nodes ]

In [42]:
physeq.23s.genus = tax_glom(physeq.23s, "Genus")

In [43]:
physeq.23s.genus

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 57 taxa and 23 samples ]
sample_data() Sample Data:       [ 23 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 57 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 57 tips and 56 internal nodes ]

In [44]:
# build a prevalence table for filtering
prevalencedf = apply(X = otu_table(physeq.23s.genus),
                     MARGIN = 1,
                     FUN = function(x){sum(x > 0)})

prevalencedf = data.frame(Prevalence = prevalencedf,
                          TotalAbundance = taxa_sums(physeq.23s.genus))

In [45]:
# filter out features that are not presnet in 10% of all samples
prevalenceThreshold = 0.10 * nsamples(physeq.23s.genus)
prevalenceThreshold
keepTaxa = rownames(prevalencedf)[(prevalencedf$Prevalence >= prevalenceThreshold)]
length(keepTaxa)
#keepTaxa

[1] 2.3

[1] 26

In [46]:
physeq.23s.genus.prevalent = prune_taxa(keepTaxa, physeq.23s.genus)
physeq.23s.genus.prevalent

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 26 taxa and 23 samples ]
sample_data() Sample Data:       [ 23 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 26 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 26 tips and 25 internal nodes ]

### Relative

In [47]:
physeq.23s.genus.prevalent.rel = transform_sample_counts(physeq.23s.genus.prevalent, function(x) 100 * x/sum(x))
sample_sums(physeq.23s.genus.prevalent.rel)

T1R1 T1R4 T1R7 T2R1 T2R4 T2R5 T2R7 T2R9 T3R1 T3R4 T3R5 T3R7 T3R9 T4R1 T4R4 T4R5 
 100  100  100  100  100  100  100  100  100  100  100  100  100  100  100  100 
T4R7 T4R9 T5R1 T5R4 T5R5 T5R7 T5R9 
 100  100  100  100  100  100  100

In [48]:
# reduce the name length: p.rel.mdf = physeq.whole.genus.prevelant.rel.mdf
p.rel.mdf = subset_samples(physeq.23s.genus.prevalent.rel, Material == "MDF")
# filter out features that are not present in all samples
p.rel.mdf = prune_taxa(taxa_sums(p.rel.mdf) > 0, p.rel.mdf)
# filter out fatures that are NA at Genus level
p.rel.mdf = subset_taxa(p.rel.mdf, !Genus == "g__unidentified")
p.rel.mdf

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 16 taxa and 14 samples ]
sample_data() Sample Data:       [ 14 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 16 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 16 tips and 15 internal nodes ]

In [49]:
write.csv(otu_table(p.rel.mdf), "ITS-rel-mdf.csv")
write.csv(tax_table(p.rel.mdf), "ITS-taxonomy-mdf.csv")

In [50]:
p.rel.gyp = subset_samples(physeq.23s.genus.prevalent.rel, Material == "Gypsum")
p.rel.gyp = prune_taxa(taxa_sums(p.rel.gyp) > 0, p.rel.gyp)
p.rel.gyp = subset_taxa(p.rel.gyp, !Genus == "g__unidentified")
p.rel.gyp

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 18 taxa and 9 samples ]
sample_data() Sample Data:       [ 9 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 18 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 18 tips and 17 internal nodes ]

In [51]:
write.csv(otu_table(p.rel.gyp), "ITS-rel-gyp.csv")
write.csv(tax_table(p.rel.gyp), "ITS-taxonomy-gyp.csv")

### Quantitative

In [52]:
# using previous objects, and convert relative abundance to quantitative abundance
#sample_data(p.rel.mdf)
count.mdf = as.data.frame(sample_data(p.rel.mdf))$Counts
p.quan.mdf = p.rel.mdf
otu_table(p.quan.mdf) = rel_to_quan(p.quan.mdf, count.mdf)

In [53]:
write.csv(otu_table(p.quan.mdf), "ITS-quan-mdf.csv")

In [55]:
#sample_data(p.rel.gyp)
count.gyp = as.data.frame(sample_data(p.rel.gyp))$Counts
p.quan.gyp = p.rel.gyp
otu_table(p.quan.gyp) = rel_to_quan(p.quan.gyp, count.gyp)

In [56]:
write.csv(otu_table(p.quan.gyp), "ITS-quan-gyp.csv")

In [57]:
count.gyp

[1]    1018.769   25284.000    4966.500 1045376.920   40260.530 3090887.460
[7]  122954.316 3431747.820 1718805.060